In [1]:
from dask.distributed import Client
client = Client(n_workers=4)



In [81]:
from pathlib import Path
import sys
import os
sys.path.append(os.path.abspath(os.path.join("..", "./data")))
import time
import dask.dataframe as dd
import pandas as pd
import numpy as np
import cupy as cp
from sklearn.preprocessing import scale
from statsmodels.nonparametric.kde import KDEUnivariate

In [3]:
filename="2022100100_imps.gz"
filepath=Path('C:\\Users\\kingo\\OneDrive\\Desktop\\Gray Swan\\data', filename)

In [13]:
df0=dd.read_csv(filepath, blocksize=None)

In [23]:
columns=df0.columns
new_columns={columns[i]: "col_"+str(i) for i in range(len(columns))}
df0=df0.rename(columns=new_columns)

In [63]:
def reading_all_zip_files(dir_path):
    res=[]
    # Iterate directory
    for path in os.listdir(dir_path):
    # check if current path is a file
        if os.path.isfile(os.path.join(dir_path, path)):
            res.append(path)
    return res   

def dask_data_frame_creation(dir_path):
    #identify all files under dir_path
    file_names=reading_all_zip_files(dir_path)
    dask_master={}
    #create a dask dataframe for each file
    for item in file_names:
        file_path=Path(dir_path, item)
        #read dask dataframe
        df=dd.read_csv(file_path, blocksize=None, dtype={'169843': 'object',
                                                           '84003': 'object',
                                                           'Unnamed: 36': 'object',
                                                           'Unnamed: 66': 'object',
                                                           'Unnamed: 67': 'object',
                                                           'nan': 'object',
                                                           'nan.1': 'object',
                                                           'nan.2': 'object'})
        #rename columns
        columns=df.columns
        new_columns={columns[i]: "col_"+str(i) for i in range(len(columns))}
        df=df.rename(columns=new_columns)
        #append dask data frame to master data structure
        dask_master[item]=df
    return dask_master
    

In [64]:
current_time = time.time()
dask_master=dask_data_frame_creation('C:\\Users\\kingo\\OneDrive\\Desktop\\Gray Swan\\data')
total_time=time.time()-current_time
print(total_time)

102.08583378791809


In [34]:
dask_master

{'2022100100_imps.gz': Dask DataFrame Structure:
                col_0  col_1   col_2   col_3   col_4  col_5    col_6   col_7    col_8    col_9   col_10   col_11   col_12 col_13 col_14 col_15  col_16   col_17   col_18 col_19  col_20  col_21  col_22  col_23 col_24   col_25   col_26   col_27  col_28   col_29   col_30  col_31  col_32  col_33 col_34   col_35   col_36 col_37 col_38 col_39 col_40   col_41 col_42   col_43   col_44  col_45   col_46   col_47   col_48 col_49  col_50  col_51   col_52   col_53 col_54  col_55   col_56  col_57  col_58  col_59  col_60   col_61   col_62   col_63   col_64 col_65   col_66   col_67  col_68   col_69 col_70 col_71 col_72  col_73   col_74  col_75
 npartitions=1                                                                                                                                                                                                                                                                                                              

In [88]:
df1=dask_master['2022100100_imps.gz']
#x00=pd.plotting.autocorrelation_plot(np.array(df1["col_1"]))
c1=df1.col_1
c5=df1.col_5
c6=df1.col_6
c10=df1.col_10
c11=df1.col_11
c12=df1.col_12

c15=df1.col_15
c35=df1.col_35
c48=df1.col_48
c49=df1.col_49
c69=df1.col_69


df1.count()

Dask Series Structure:
npartitions=1
col_0    int64
col_9      ...
dtype: int64
Dask Name: dataframe-count-agg, 4 tasks

In [90]:
np.corrcoef([c1, c5, c6, c15, c35, c48, c49, c69])

array([[ 1.        ,  0.42197834,  0.08364204,  0.25047536, -0.41007611,
        -0.13739317,  0.29316852,  0.23896975],
       [ 0.42197834,  1.        , -0.13713069,  0.25307137,  0.05792697,
        -0.39398793,  0.36399791,  0.13293034],
       [ 0.08364204, -0.13713069,  1.        , -0.21865773, -0.39446645,
         0.51332706, -0.17838858, -0.05590831],
       [ 0.25047536,  0.25307137, -0.21865773,  1.        , -0.1578332 ,
        -0.00723956,  0.93438269,  0.38429379],
       [-0.41007611,  0.05792697, -0.39446645, -0.1578332 ,  1.        ,
        -0.41993719, -0.2724532 , -0.10079683],
       [-0.13739317, -0.39398793,  0.51332706, -0.00723956, -0.41993719,
         1.        , -0.00673665, -0.01729443],
       [ 0.29316852,  0.36399791, -0.17838858,  0.93438269, -0.2724532 ,
        -0.00673665,  1.        ,  0.41367625],
       [ 0.23896975,  0.13293034, -0.05590831,  0.38429379, -0.10079683,
        -0.01729443,  0.41367625,  1.        ]])

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


In [67]:
r3 = cp.corrcoef(c1)

CUDARuntimeError: cudaErrorUnknown: unknown error

In [78]:
df_NA=df1.isna().sum().reset_index().rename(columns={"index":"column_name", 0:"number_of_NA"})

In [80]:
df_NA[df_NA["number_of_NA"]!=0].compute()

,column_name,number_of_NA
16,col_16,76982
31,col_31,42628
32,col_32,24094
33,col_33,3220
36,col_36,141315
50,col_50,76982
61,col_61,397226
62,col_62,272013
63,col_63,552629
64,col_64,636954


In [82]:
def find_outliers_tukey(x):

    q1 = np.percentile(x, 25)
    q3 = np.percentile(x, 75)
    iqr = q3-q1
    floor = q1 - 1.5*iqr
    ceiling = q3 + 1.5*iqr
    outlier_indeces = list(x.index[(x < floor) | (x > ceiling)])
    outlier_values = list(x[outlier_indeces])

    return outlier_values

def find_outliers_kde(x):
    x_scaled = scale(list(map(float,x)))
    kde = KDEUnivariate(x_scaled)
    kde.fit(bw='scott',fft=True)
    pred = kde.evaluate(x_scaled)

    n = sum(pred<0.05)
    outlier_ind = np.asarray(pred).argsort()[:n]
    outlier_value = np.asarray(x)[outlier_ind]

    return outlier_value

In [87]:
#df1.col_29.apply(lambda x: find_outliers_kde(x))
def kde_outlier_map(column):
    df1[column]=df1[column].apply(lambda x: find_outliers_kde(x))
    return df1[column]

df1.map_partitions(kde_outlier_map, meta="col_29").compute()

NotImplementedError:         col_0  col_1     col_2       col_3     col_4  col_5       col_6  \
0           2    429  prod_429  NBCU_Crexi        -1   1530    52654.77   
1           2    429  prod_429  NBCU_Crexi        -1   1530    52654.77   
2           2    429  prod_429  NBCU_Crexi        -1   1530    52654.77   
3           2    429  prod_429  NBCU_Crexi        -1   1530    52654.77   
4           2    429  prod_429  NBCU_Crexi        -1   1530    52654.77   
...       ...    ...       ...         ...       ...    ...         ...   
640809      2    540  prod_540      Frndly  Pluto TV   1770  1827118.91   
640810      2    540  prod_540      Frndly  Pluto TV   1770  1827118.91   
640811      2    540  prod_540      Frndly  Pluto TV   1770  1827118.91   
640812      2    540  prod_540      Frndly  Pluto TV   1770  1827118.91   
640813      2    540  prod_540      Frndly  Pluto TV   1770  1827118.91   

                                               col_7  col_8   col_9  ...  \
0 